In [1]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import warnings
import argparse
import pickle
import cv2
import os
import sys
import glob
import matplotlib
matplotlib.use("Agg")

In [2]:
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [4]:
args = {
    "dataset": "./SCVD_mini/5x3",
    "model": "./MoBiLSTM5x3/models/MoBiLSTM_5x3.h5",
    "label-bin": "./MoBiLSTM5x3/label-bin/MoBiLSTM_5x3.pickle",
    "epochs": 30,
    "plot": "./MoBiLSTM5x3/plots/MoBiLSTM_5x3.png"
}

In [5]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["Violence", "NonViolence", "WeaponViolence"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print('-'*100)
print("[INFO] loading images...")
print('-'*100)
from imutils import paths
imagePaths = list(paths.list_images(args["dataset"]))
print(imagePaths)
data = []
labels = []

----------------------------------------------------------------------------------------------------
[INFO] loading images...
----------------------------------------------------------------------------------------------------
['./SCVD_mini/5x3\\NonViolence\\nv1_merge_0.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_1.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_2.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_3.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_4.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_5.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_6.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_7.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_8.jpg', './SCVD_mini/5x3\\NonViolence\\nv1_merge_9.jpg', './SCVD_mini/5x3\\NonViolence\\nv2_merge_0.jpg', './SCVD_mini/5x3\\NonViolence\\nv2_merge_1.jpg', './SCVD_mini/5x3\\NonViolence\\nv2_merge_2.jpg', './SCVD_mini/5x3\\NonViolence\\nv2_merge_3.jpg', './SCVD_mini/5x3\\NonViolence\\nv2_merge_4.jpg', './SCVD_mini/5x3\\NonViolence\\nv2_me

In [6]:
# loop over the image paths
for imagePath in tqdm(imagePaths[::]):
    # imagePath : file name ex) V_123
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2] # Violence / NonViolence

    # if the label of the current image is not part of of the labels
    # are interested in, then ignore the image
    if label not in LABELS:
        continue

    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [00:59<00:00,  1.48it/s]


In [7]:
# convert the data and labels to NumPy arrays
dt = np.array(data)
lbls = np.array(labels)


# perform one-hot encoding on the labels
lb = LabelBinarizer()
lbls_fit = lb.fit_transform(lbls)
#lbls_cat = to_categorical(lbls_fit, num_classes=3)

# partition the data into training and testing splits using 70% of
# the data for training and the remaining 30% for testing
(trainX, testX, trainY, testY) = train_test_split(dt, lbls_fit, test_size=0.2, stratify=lbls_fit, random_state=33)

In [8]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    #rotation_range=30,
    zoom_range=0.15,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [9]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [43]:
# load the InceptionV3 network, ensuring the head FC layer sets are left
# off
mobilenet = MobileNetV2(
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=3,
    classifier_activation="softmax")

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
mobilenet.trainable = True

In [44]:
from keras.models import Sequential
from keras.layers import *


def create_model():
 
    model = Sequential()

    ########################################################################################################################
    
    #Specifying Input to match features shape
    model.add(Input(shape = (15, 224, 224, 3)))
    
    # Passing mobilenet in the TimeDistributed layer to handle the sequence
    model.add(TimeDistributed(mobilenet))
    
    model.add(Dropout(0.25))
                                    
    model.add(TimeDistributed(Flatten()))

    
    lstm_fw = LSTM(units=32)
    lstm_bw = LSTM(units=32, go_backwards = True)  

    model.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
    
    model.add(Dropout(0.25))

    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.25))
    
    
    model.add(Dense(3, activation = 'softmax'))
 
    ########################################################################################################################
 
    model.summary()
    
    return model

In [45]:
# Constructing the Model
MoBiLSTM_model = create_model()

# Plot the structure of the contructed LRCN model.
plot_model(MoBiLSTM_model, to_file = 'MobBiLSTM_model_structure_plot.png', show_shapes = True, show_layer_names = True)

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print('-'*100)
print("[INFO] compiling model...")
print('-'*100)
opt = SGD(lr=1e-3, momentum=0.9, decay=1e-4 / args["epochs"])
MoBiLSTM_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print(MoBiLSTM_model.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_17 (TimeDi  (None, 15, 3)            2261827   
 stributed)                                                      
                                                                 
 dropout_48 (Dropout)        (None, 15, 3)             0         
                                                                 
 time_distributed_18 (TimeDi  (None, 15, 3)            0         
 stributed)                                                      
                                                                 
 bidirectional_8 (Bidirectio  (None, 64)               9216      
 nal)                                                            
                                                                 
 dropout_49 (Dropout)        (None, 64)                0         
                                                     

In [46]:
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print('-'*100)
print("[INFO] training head...")
print('-'*100)
H = MoBiLSTM_model.fit(trainAug.flow(trainX, trainY, batch_size=32),
            steps_per_epoch=len(trainX) // 32,
            validation_data=valAug.flow(testX, testY),
            validation_steps=len(testX) // 32,
            epochs=args["epochs"])

----------------------------------------------------------------------------------------------------
[INFO] training head...
----------------------------------------------------------------------------------------------------
Epoch 1/30


ValueError: in user code:

    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Python310\lib\site-packages\keras\engine\input_spec.py", line 232, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer "sequential_12" "                 f"(typ

In [ ]:
# evaluate the network
print('-'*100)
print("[INFO] evaluating network...")
print('-'*100)
predictions = MoBiLSTM_model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

In [ ]:
# plot the training loss and accuracy
print('-'*100)
print("[INFO] plot the training loss and accuracy...")
print('-'*100)
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

In [ ]:
# serialize the model to disk
print('-'*100)
print("[INFO] serializing network...")
print('-'*100)
MoBiLSTM_model.save(args["model"])

In [ ]:
# serialize the label binarizer to disk
f = open(args["label-bin"], "wb")
f.write(pickle.dumps(lb))
f.close()